In [1]:
import re
import tweepy
import pandas as pd
from tweepy import OAuthHandler
import yaml
# from http.client import IncompleteRead
# import threading 
# import datetime 
    
    
with open('credential1.yaml') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']
access_token = config['access_token']
access_secret = config['access_secret']
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#### Collect tweets with hashtag `#30mapchallenges`

In [2]:
def collect_tweets(search_words, since):
    #tweets
    tweet_id = []
    tweet_text = []
    language = []
    tweet_created_at = []
    retweet_count = []
    favorite_count= []
    hashtags= []
    #users
    user_id = []
    user_name = []
    user_screen_name = []
    user_loc = []
    user_follower = []
    friends_count = []
    #media
    media = []
    for tweet in tweepy.Cursor(api.search,
                               q=search_words,
                               since= since, 
                               include_entities=True, 
                               tweet_mode="extended").items():
        #tweets
        tweet_id.append(tweet.id)
        tweet_text.append(tweet.full_text)
        language.append(tweet.lang)
        tweet_created_at.append(tweet.created_at)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
        try:
            hashtags.append(tweet.entities.get('hashtags', []))
        except KeyError:
            pass
#         hashtags.append(tweet.entities.hashtags)
        #users
        user_id.append(tweet.user.id)
        user_name.append(tweet.user.name)
        user_screen_name.append(tweet.user.screen_name)
        user_loc.append(tweet.user.location)
        user_follower.append(tweet.user.followers_count)
        friends_count.append(tweet.user.friends_count)
        #media
        try:
            media.append(tweet.entities.get('media')[0].get('media_url'))
#             print(tweet.entities.get('media'))
        except Exception as e:
            media.append(None)
        #print([tweet.user.name,tweet.created_at,tweet.text])
    df = pd.DataFrame({'tweet_id':tweet_id,
                        'tweet_text':tweet_text,
                        'language':language,
                        'tweet_created_at':tweet_created_at,
                        'retweet_count':retweet_count,
                        'favorite_count':favorite_count,
                        'hashtags':hashtags,
                        'user_id':user_id,
                        'user_name':user_name,
                        'user_screen_name':user_screen_name,
                        'user_loc':user_loc,
                        'user_follower':user_follower,
                        'friends_count':friends_count,
                        'media':media})
            
    return df

In [3]:
search_words = "#30DayMapChallenge -filter:retweets"
since = "2020-10-30"

df = collect_tweets(search_words, since)

df = df[(df.tweet_created_at >= '2020-11-03 00:00:00') & 
   (df.tweet_created_at <= '2020-11-04 00:00:00')]

# df1 = df[df['tweet_text'].str.contains("points") | df['tweet_text'].str.contains("day 1")]

df = df[df['media'].notnull()]
df['tweet_url'] = df['tweet_text'].str.extract('(?P<url>https?://[^\s]+)', expand=False).str.strip()


# df.to_csv(f"./data/day3.csv")

In [4]:
df_day2_late = df[df['tweet_text'].str.contains('lines', case=False) & df['tweet_text'].str.contains('day 2', case=False)]
# df_day2_late.to_csv(f"./data/day2_late.csv")

In [6]:
# exclude late entry

## Overview

In [188]:
# df = pd.read_csv("./data/day1.csv")

In [122]:
df.user_loc

163                     
164    Neo SCL / Neo BCN
165                 Wien
166     Montréal, Québec
169                     
             ...        
704                     
705                  y,x
706        Leicester, UK
707                     
708                 Iraq
Name: user_loc, Length: 456, dtype: object

In [7]:
df[['retweet_count','favorite_count', 'user_follower', 'friends_count']].describe()

,retweet_count,favorite_count,user_follower,friends_count
count,404.000000,404.000000,404.000000,404.000000
mean,2.383663,11.928218,1556.784653,749.037129
std,2.790240,19.890327,5664.620914,919.315044
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,96.000000,178.250000
50%,1.000000,6.000000,293.000000,394.000000
75%,3.000000,13.000000,897.750000,978.000000
max,25.000000,238.000000,80331.000000,5454.000000


## Top favorited `#30mapchallenges` post

In [8]:
df_fav15 = df.sort_values(by='favorite_count', ascending=False).head(15)
df_fav15 = df_fav15[['user_screen_name', 'retweet_count', 'favorite_count', 'media']]
df_fav15['user_screen_name'] = '**[@' + df['user_screen_name'] +']('+ df['tweet_url']+')**'
df_fav15['media'] = '![](' + df['media'] +')'
df_fav15

,user_screen_name,retweet_count,favorite_count,media
556,**[@tjukanov](https://t.co/uLN9PJeg1k)**,21,238,![](http://pbs.twimg.com/media/El4Qv6tW0AEoreL...
356,**[@Helenmckmck](https://t.co/pPbs6Py9ZI)**,21,151,![](http://pbs.twimg.com/media/El5-yUKWkAIS7OD...
523,**[@kennethfield](https://t.co/O5is4PtFCb)**,17,138,![](http://pbs.twimg.com/media/El285DrUcAA_NEv...
491,**[@dr_xeo](https://t.co/uUujVgV2ql)**,25,92,![](http://pbs.twimg.com/media/El4w90wXEAAu2gp...
576,**[@thekuhugupta](https://t.co/PBBwm0kMKT)**,6,89,![](http://pbs.twimg.com/media/El3v1L8VMAEMWbu...
176,**[@moaimx](https://t.co/aSJFhEp7za)**,17,78,![](http://pbs.twimg.com/media/El7kZTGXEAIHeIy...
380,**[@darenw](https://t.co/IVnh4RDG2q)**,5,75,![](http://pbs.twimg.com/media/El5z7aVXYAIQl6N...
393,**[@rajbhagatt](https://t.co/PqM78uwwOA)**,2,63,![](http://pbs.twimg.com/media/El5vxefVMAAeOPJ...
344,**[@evelynuuemaa](https://t.co/gM1T6SJAdj)**,1,55,![](http://pbs.twimg.com/media/El6DH2aXIAE4-an...
588,**[@ibltzr](https://t.co/HN0CbDLZYK)**,7,54,![](http://pbs.twimg.com/ext_tw_video_thumb/13...


In [9]:
from tabulate import tabulate

print(tabulate(df_fav15, headers='keys', tablefmt='github'))

|     | user_screen_name                               |   retweet_count |   favorite_count | media                                                                                        |
|-----|------------------------------------------------|-----------------|------------------|----------------------------------------------------------------------------------------------|
| 556 | **[@tjukanov](https://t.co/uLN9PJeg1k)**       |              21 |              238 | ![](http://pbs.twimg.com/media/El4Qv6tW0AEoreL.jpg)                                          |
| 356 | **[@Helenmckmck](https://t.co/pPbs6Py9ZI)**    |              21 |              151 | ![](http://pbs.twimg.com/media/El5-yUKWkAIS7OD.jpg)                                          |
| 523 | **[@kennethfield](https://t.co/O5is4PtFCb)**   |              17 |              138 | ![](http://pbs.twimg.com/media/El285DrUcAA_NEv.jpg)                                          |
| 491 | **[@dr_xeo](https://t.co/uUujVgV2ql)**         

## Top retweeted `#30mapchallenges` post

In [10]:
df_rt15 = df.sort_values(by='retweet_count', ascending=False).head(15)
df_rt15 = df_rt15[['user_screen_name', 'retweet_count', 'favorite_count', 'media']]
df_rt15['user_screen_name'] = '**[@' + df['user_screen_name'] +']('+ df['tweet_url']+')**'
df_rt15['media'] = '![](' + df['media'] +')'
df_rt15

,user_screen_name,retweet_count,favorite_count,media
491,**[@dr_xeo](https://t.co/uUujVgV2ql)**,25,92,![](http://pbs.twimg.com/media/El4w90wXEAAu2gp...
356,**[@Helenmckmck](https://t.co/pPbs6Py9ZI)**,21,151,![](http://pbs.twimg.com/media/El5-yUKWkAIS7OD...
556,**[@tjukanov](https://t.co/uLN9PJeg1k)**,21,238,![](http://pbs.twimg.com/media/El4Qv6tW0AEoreL...
523,**[@kennethfield](https://t.co/O5is4PtFCb)**,17,138,![](http://pbs.twimg.com/media/El285DrUcAA_NEv...
176,**[@moaimx](https://t.co/aSJFhEp7za)**,17,78,![](http://pbs.twimg.com/media/El7kZTGXEAIHeIy...
427,**[@geodatascience](https://t.co/b4Rm1s3qua)**,15,35,![](http://pbs.twimg.com/media/El4_XQrXUAYJGTU...
347,**[@jakub_nowosad](https://t.co/qnzLDxIOjC.)**,12,34,![](http://pbs.twimg.com/media/El6DWvaWMAES9R8...
160,**[@Valtzen](https://t.co/3SKLXYljS9)**,10,47,![](http://pbs.twimg.com/media/El7uBX8WkAA7Ujo...
455,**[@dickoah](https://t.co/AFj9vaRwmH)**,10,32,![](http://pbs.twimg.com/media/El5LgkgW0AEZPLX...
589,**[@PratapVardhan](https://t.co/Mt4CqRkNlL)**,10,40,![](http://pbs.twimg.com/ext_tw_video_thumb/13...


In [11]:
print(tabulate(df_rt15, headers='keys', tablefmt='github'))

|     | user_screen_name                                |   retweet_count |   favorite_count | media                                                                                        |
|-----|-------------------------------------------------|-----------------|------------------|----------------------------------------------------------------------------------------------|
| 491 | **[@dr_xeo](https://t.co/uUujVgV2ql)**          |              25 |               92 | ![](http://pbs.twimg.com/media/El4w90wXEAAu2gp.jpg)                                          |
| 356 | **[@Helenmckmck](https://t.co/pPbs6Py9ZI)**     |              21 |              151 | ![](http://pbs.twimg.com/media/El5-yUKWkAIS7OD.jpg)                                          |
| 556 | **[@tjukanov](https://t.co/uLN9PJeg1k)**        |              21 |              238 | ![](http://pbs.twimg.com/media/El4Qv6tW0AEoreL.jpg)                                          |
| 523 | **[@kennethfield](https://t.co/O5is4PtFCb)

### Send tweet via API

In [3]:
api.update_status("Look, I'm tweeting from #Python")


Status(_api=<tweepy.api.API object at 0x0000011E60202A48>, _json={'created_at': 'Mon Nov 02 22:53:36 +0000 2020', 'id': 1323397878983856130, 'id_str': '1323397878983856130', 'text': "Look, I'm tweeting from #Python", 'truncated': False, 'entities': {'hashtags': [{'text': 'Python', 'indices': [24, 31]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://hn303.github.io/CamLandEc-RM03/" rel="nofollow">Lonon_REST</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 528087151, 'id_str': '528087151', 'name': 'Haifeng Niu', 'screen_name': 'niu_haifeng', 'location': 'Cambridge, England', 'description': 'PhD candidate in Lab of Interdisciplinary Spatial Analysis, Department of Land Economy, University of Cambridge @CambridgeLandEc', 'url': 'https://t.co/nuaPhizg9Y', 'entities': {'url': {'urls': [{'url': 'https://t.co/nuaPhizg9Y', 'expanded_